In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, TextIteratorStreamer
from threading import Thread
import time

/home/huy/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("./checkpoint")
model = AutoModelForCausalLM.from_pretrained("./checkpoint")

In [3]:
('bốn chữ', 'năm chữ', 'sáu chữ', 'bảy chữ', 'tám chữ', 'lục bát', 'song thất lục bát')

('bốn chữ',
 'năm chữ',
 'sáu chữ',
 'bảy chữ',
 'tám chữ',
 'lục bát',
 'song thất lục bát')

In [4]:
sum(i.numel() for i in model.parameters())

124442112

In [5]:
start_word = "Ngày mai"
gender = "lục bát"
prompt = f"{tokenizer.bos_token}{gender}{tokenizer.sep_token}{start_word}"
input_ids = tokenizer.encode(prompt, return_tensors='pt')
# generated = model.generate(
#                 input_ids,
#                 pad_token_id=tokenizer.eos_token_id,
#                 do_sample=True,
#                 max_length=70,
#                 top_k=4,
#                 num_beams=5,
#                 no_repeat_ngram_size=2,
#                 num_return_sequences=1
#             )
# text = tokenizer.decode(generated[0].tolist())
# print(text)

In [ ]:
streamer = TextIteratorStreamer(tokenizer)
generation_kwargs = dict(inputs=input_ids, streamer=streamer, max_new_tokens=70, pad_token_id=tokenizer.eos_token_id,do_sample=True, top_k=4,no_repeat_ngram_size=2)
thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()
generated_text = ""
total_sep = 0
for new_text in streamer:
    total_sep += 1
    if total_sep == 50:
        break
    generated_text += new_text
    time.sleep(0.1)
    text = new_text.replace("<|startoftext|>", "").replace(gender, "").replace("<|sep|>", "\n").replace("<|endoftext|>", "\n")
    if text == "\n":
        print("aaaaaaaaaa")
    if "\n\n" in text:
        index = text.find("\n\n")
        print(text[:index])
        break
    else:
        if text.startswith('\n'):
            text = text.replace('\n', '').capitalize()
        if '\n' in text:
            t = text.split('\n')
            u = t[1].capitalize()
            text = '\n'.join([t[0], u])
        print(text, end="")

Ngày mai ngày mốt em về
Anh đi để lại những lời thương em
Nhớ thương anh biết nói sao
Để cho em biết tình nào anh trao


In [ ]:
text

'ngang\nem '

In [ ]:
"".capitalize

In [14]:
start_word = "Ngày mai"
gender = "lục bát"
prompt = f"{tokenizer.bos_token}{gender}{tokenizer.sep_token}{start_word}"
input_ids = tokenizer.encode(prompt, return_tensors='pt')
streamer = TextIteratorStreamer(tokenizer)
generation_kwargs = dict(inputs=input_ids, streamer=streamer, max_new_tokens=70, pad_token_id=tokenizer.eos_token_id,do_sample=True, top_k=4,no_repeat_ngram_size=2)
thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()
generated_text = ""
total_sep = 0
for new_text in streamer:
    generated_text += new_text
    # if generated_text not in prompt:
    #     continue
    print(new_text)


<|startoftext|>lục bát<|sep|>Ngày 
mai 
ngày 
lễ 
tình 


yêu<|endoftext|>em 
về 
bên 
ấy 
có 
nhiều 
nhớ 


mong<|endoftext|>anh 
về 
với 
những 
mong 
mong 


về<|endoftext|>ngày 
vui 
anh 
có 
về 
chung 
một 




nhà<|endoftext|><|endoftext|><|sep|><|endoftext|> 
võ 
nghệ 
nghệ 
là 
tên 
gọi 
của 

em<|endoftext|> 
thương 
thương 
em 
lắm 
em 



ạ<|endoftext|><|startoftext|><|endoftext|> 
anh 
ơi 
anh 
hãy 
nói 
lời 
này 


nghe<|endoftext|>trong 
tim 
em 
đó 
anh 
yêu 
em 
nhiều 
lắm
